# Домашняя работа

In [95]:
import re
import nltk
import requests
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.metrics import accuracy_score
from bs4 import BeautifulSoup

nltk.download("punkt")
morph = MorphAnalyzer()

[nltk_data] Downloading package punkt to /home/marynepo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Пройдемся по страницам с отзывами в кинопоиске, скачаем тексты отзывов, деля их на положительные и отрицательные.

In [86]:
revs = {'pos':[], 'neg': []}
response = requests.get('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/bad/perpage/50/', headers={'User-agent':'Mozilla/5.0'}).text
soup = BeautifulSoup(response,'html.parser')
for l in soup.find_all('span', {'class':'_reachbanner_', 'itemprop':'reviewBody'})[:50]:
    revs['neg'].append(l.get_text())

In [88]:
response = requests.get('https://www.kinopoisk.ru/film/401177/reviews/ord/date/status/good/perpage/50/', headers={'User-agent':'Mozilla/5.0'}).text
soup = BeautifulSoup(response,'html.parser')
for l in soup.find_all('span', {'class':'_reachbanner_', 'itemprop':'reviewBody'})[:50]:
    revs['pos'].append(l.get_text())

Отделим тестовую выборку

In [101]:
X_test = revs['pos'][40:] + revs['neg'][40:]
Y_test = ['pos']*10 + ['neg']*10

Токенизируем и лемматизируем отзывы и составляем словарь со словами, которые встречаются либо только в положительных, либо только в отрицательных отзывах.

In [141]:
def words_to_voc(text):
    
    tokens = nltk.word_tokenize(text.lower())
    words = []
    for t in tokens:
        if str(morph.parse(',')[0].tag) is 'PNCT':
            words.append(morph.parse(t)[0].normal_form)
            
    return words
            
def make_voc(texts):
    
    pos_words = Counter()
    neg_words = Counter()
    
    for t in texts['pos'][:40]:
        pos_words.update(words_to_voc(t))
    for t in texts['neg'][:40]:
        neg_words.update(words_to_voc(t))
        
    pos_w = set(pos_words.keys()) - set(neg_words.keys())
    neg_w = set(neg_words.keys()) - set(pos_words.keys())
    
    words = {'pos':[], 'neg': []}
    
    for w in pos_w:
        if pos_words[w] > 2:
            words['pos'].append(w)
            
    for w in neg_w:
        if neg_words[w] > 2:
            words['neg'].append(w)
    
    return words

voc = make_voc(revs)

Создадим функцию, которая предсказывает, положительный ли отзыв или отрицательный.

In [144]:
def mark_text(text):
    words = words_to_voc(text) #можно еще сделать сет из этого, чтобы не учитывать повторение, но у меня так accuracy хуже
    n = 0
    p = 0
    for w in words:
        if w in voc['pos']:
            p += 1
        elif w in voc['neg']:
            n += 1
    if p >= n:
        return 'pos'
    else:
        return 'neg'

И проверим accuracy.

In [145]:
def check_acc(x, y):
    res = []  
    for text in x:
        res.append(mark_text(text))
    print("Accuracy: %.4f" % accuracy_score(res, y))
    
check_acc(X_test, Y_test)

Accuracy: 0.8500


Улучшение:

1) Учитывать частотность слов и добавить веса в соответствии с ними

2) Посмотреть на n-граммы